## 1. Get the data

1. Copy the **Credentials / CLI Snippets**.

It would look like...

In [ ]:
export AWS_DEFAULT_REGION=us-east-1
export AWS_ACCESS_KEY_ID=ASIAT27S2FsT6RHKVT7D
export AWS_SECRET_ACCESS_KEY=Fbsb6282OBkMSA/iwpIwvs+crbZ+E2DZOAajmIkk
export AWS_SESSION_TOKEN=FwoGZXIvYXdzEIr//////////wEaDEm145FRWPTP33mTeiKuAc2EnC123DtsExywKl7Vixzwg0hqDHgihlFvpgWUjCGhDVIM+PGSp8HwZ4utkcqV0x0asVs/CFdRqQh5OmYPW+uUBr+7DKcByH9ivJ+ze2lLkqkvYBvP8mHeF2ny6fJFufZmKCQy6k4D/jr9tRurufF989XupYWaPBNZ7gTdi7cQOzeW9tquLP+bhUT8hjR11nA+G7VwNTZEY15+WHLaeBMEf73cVORP3sn33FY0yyjptOzzBTIt8xzZKY+fSDlQuQ90GnsnU8NFHTF27t7rAc7fYid4bf2zy++5bUIA3VB/+P8a

2. Start the cluster with m3.xlarge

3. Inside the Notebook open a Terminal

In [ ]:
aws s3 ls

In [ ]:
curl https://aws-emr-resources-25032020-us-east-1.s3.amazonaws.com/taxidata.csv | aws s3 cp - s3://<bucket-name>/taxidata.csv

In [ ]:
aws s3 ls s3://<bucket-name>

## 2. The spark code inside the Notebook

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, isnan, when, trim

import time

In [ ]:
%%info

**s3distcp**

https://docs.aws.amazon.com/emr/latest/ReleaseGuide/UsingEMR_s3distcp.html

In [ ]:
green_taxi = spark.read.format("s3selectCSV").option("header", "true").option("inferSchema", "true").\
load("s3://<bucket_name>/taxidata.csv")

In [ ]:
green_taxi.columns

In [ ]:
type(green_taxi)

In [ ]:
print((green_taxi.count(), len(green_taxi.columns)))

# Do pre-processing in Spark SQL

In [ ]:
green_taxi.createOrReplaceTempView("taxi")

In [ ]:
start_time = time.time()
sqlDF = spark.sql("SELECT PULocationID, DOLocationID, passenger_count, trip_distance, RatecodeID, \
                    total_amount, payment_type, trip_type, tip_amount, fare_amount, \
                    ROUND(CAST(tip_amount/fare_amount AS DOUBLE), 4) as tip_percent, \
                    CAST(from_unixtime(unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'yyyy') AS INT) as pickup_year, \
                    CAST(from_unixtime(unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'MM') AS INT) as pickup_month,\
                    CAST(from_unixtime(unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'dd') AS INT) as pickup_day, \
                    CAST(from_unixtime(unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'hh') AS INT) as pickup_hour, \
                    CAST(from_unixtime(unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'mm') AS INT) as pickup_minute, \
                    CAST(from_unixtime(unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'yyyy') AS INT) as dropoff_year, \
                    CAST(from_unixtime(unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'MM') AS INT) as dropoff_month,\
                    CAST(from_unixtime(unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'dd') AS INT) as dropoff_day, \
                    CAST(from_unixtime(unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'hh') AS INT) as dropoff_hour, \
                    CAST(from_unixtime(unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa'), 'mm') AS INT) as dropoff_minute, \
                    ROUND(CAST((unix_timestamp(lpep_dropoff_datetime, 'MM/dd/yyyy hh:mm:ss aa') - unix_timestamp(lpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss aa'))/360 AS DOUBLE), 4) as tripdurr \
                    FROM taxi WHERE fare_amount > 2.50")

sqlDF.createOrReplaceTempView("taxi")

print(f'Total time: {(time.time() - start_time):.3f} seconds.')

In [ ]:
sqlDF = sqlDF.na.drop()

In [ ]:
sqlDF.createOrReplaceTempView("taxi")
sqlDF = spark.sql("SELECT *, \
    ROUND(CAST(trip_distance/tripdurr AS DOUBLE), 4) as avg_speed \
    FROM taxi WHERE pickup_month in (1, 2) AND pickup_year=2017 AND tip_percent<1").na.drop()

# Use SparkML data (feature) transformers

https://spark.apache.org/docs/latest/ml-features

In [ ]:
from pyspark.sql.types import IntegerType

### Use UDF functions

In [ ]:
def check_airport_id(id):
     return int((id == 1) | (id == 2))
    
check_airport_id_udf = udf(check_airport_id, IntegerType())
sqlDF = sqlDF.withColumn("is_airport", check_airport_id_udf(sqlDF['RateCodeID']))

sqlDF.createOrReplaceTempView("taxi")

train = spark.sql("SELECT * FROM taxi WHERE pickup_month in (1) and pickup_year=2017")
train = train.drop("pickup_year").drop("pickup_month").drop("dropoff_year").drop("dropoff_month")

test = spark.sql("SELECT * FROM taxi WHERE pickup_month in (2) and pickup_year=2017")
test = test.drop("pickup_year").drop("pickup_month").drop("dropoff_year").drop("dropoff_month")

In [ ]:
#train.write.csv('nyc_taxi_train.csv')
#test.write.csv('nyc_taxi_test.csv')

# Save in Parquet files

train.write.parquet('nyc_taxi_train.parquet')
test.write.parquet('nyc_taxi_test.parquet')

In [ ]:
featuresCols = train.columns
featuresCols.remove('tip_amount')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

gbt = GBTRegressor(labelCol="tip_amount")

paramGrid = ParamGridBuilder().addGrid(gbt.maxDepth, [2]).addGrid(gbt.maxIter, [2]).build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), \
                                predictionCol=gbt.getPredictionCol())

cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid)

pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [ ]:
featuresCols

In [ ]:
start_time=time.time()
pipelineModel = pipeline.fit(train)
print(f'Total time: {(time.time() - start_time):.3f} seconds.')

In [ ]:
predictions = pipelineModel.transform(test)

rmse = evaluator.evaluate(predictions)
print("RMSE on our test set: {}".format(rmse))

### Spark tuning
https://spark.apache.org/docs/latest/tuning.html